### Importing the required libraries

In [13]:
import numpy as np
import nltk
import string
import random


* Importing and reading the corpus

In [14]:
f = open ('chatbot.txt', 'r', errors = 'ignore')
raw_doc = f.read()
raw_doc = raw_doc.lower() # convert to lower case
nltk.download('punkt') # download the punkt tokenizer, which is required for word tokenization
nltk.download('wordnet') #using the wordnet lemmatizer to reduce words to their base form
sent_tokens = nltk.sent_tokenize(raw_doc)# converts doc to list of sentences
word_tokens = nltk.word_tokenize(raw_doc) # converts doc to list of words


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Franck\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Franck\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**Example of sentance tokens**

In [ ]:
sent_tokens[:2]